In [10]:
#Summary:
#naver 환율 조회 페이지를 파싱해서 국가별 환율 정보를 조회한다.

import sys
import re
from bs4 import BeautifulSoup

import urllib
from urllib.request import Request, urlopen
from urllib.parse import quote
    
import requests as rs

In [11]:
def getPage(url):
    """
    url 정보의 내용을 조회한다.
    """
    try:
        req = Request(url)
        res = urlopen(req)
        content = res.read()
    except:
        content = ""  
    return content

In [12]:
def getExchangeOfNation(soup):
    """
    국가별 환율 정보를 dictionary로 구축한다.
    """
    dicExchange = {} # 국가별 환율정보를 저장하는 dictionary 변수
    
    alpha = '([A-Z]+)' # 국가 코드 분리 - 정규식 적용
    
    # 1, 2행의 tr은 테이블 헤더 정보이다.
    # 2행 이후부터 테이블의 컬럼값을 조회한다.
    for item in soup.table('tr')[2:]:
        
        # 정보 파싱
        nation = item('td')[0].text.strip() # 국가정보 파싱
        re_result = re.search(alpha, nation)
        nation = re_result.groups()[0]
        
        basicRateOfExchange = item('td')[1].text # 매매기준환율
        cash_buy = item('td')[2].text #  현찰 살때
        cash_sell = item('td')[3].text # 현찰 팔때
        transfer_send = item('td')[4].text # 송금 보낼 때
        transfer_receive = item('td')[5].text # 송금 받을 때
        
        dicExchange[nation] = {'basicRate':basicRateOfExchange, 'cashBuy':cash_buy, \
                   'cashSell':cash_sell, 'transferSend':transfer_send, 'transferReceive':transfer_receive}
    
    return dicExchange        


In [13]:
# naver 환율 페이지 조회
url = "http://info.finance.naver.com/marketindex/exchangeList.nhn"
    
# page 내용을 조회한다.
res = getPage(url)

soup = BeautifulSoup(res, 'html.parser')
#print(soup.prettify)
# soup.table('tr')[2]('td')[0].text.strip()
nationExchangeRate = getExchangeOfNation(soup)


{'USD': {'basicRate': '1,123.10', 'cashBuy': '1,142.75', 'cashSell': '1,103.45', 'transferSend': '1,134.10', 'transferReceive': '1,112.10'}, 'EUR': {'basicRate': '1,286.29', 'cashBuy': '1,311.88', 'cashSell': '1,260.70', 'transferSend': '1,299.15', 'transferReceive': '1,273.43'}, 'JPY': {'basicRate': '992.09', 'cashBuy': '1,009.45', 'cashSell': '974.73', 'transferSend': '1,001.81', 'transferReceive': '982.37'}, 'CNY': {'basicRate': '162.08', 'cashBuy': '170.18', 'cashSell': '153.98', 'transferSend': '163.70', 'transferReceive': '160.46'}, 'HKD': {'basicRate': '143.38', 'cashBuy': '146.20', 'cashSell': '140.56', 'transferSend': '144.81', 'transferReceive': '141.95'}, 'TWD': {'basicRate': '36.57', 'cashBuy': '41.36', 'cashSell': '34.02', 'transferSend': '0.00', 'transferReceive': '0.00'}, 'GBP': {'basicRate': '1,474.35', 'cashBuy': '1,503.39', 'cashSell': '1,445.31', 'transferSend': '1,489.09', 'transferReceive': '1,459.61'}, 'OMR': {'basicRate': '2,917.14', 'cashBuy': '3,176.76', 'cashS

In [14]:
# 환율 조회 출력
print(nationExchangeRate['JPY'])

{'basicRate': '992.09', 'cashBuy': '1,009.45', 'cashSell': '974.73', 'transferSend': '1,001.81', 'transferReceive': '982.37'}


In [23]:
basicRate = nationExchangeRate['JPY']['basicRate']
cashBuy = nationExchangeRate['JPY']['cashBuy']
cashSell = nationExchangeRate['JPY']['cashSell']
transferSend = nationExchangeRate['JPY']['transferSend']
transferReceive = nationExchangeRate['JPY']['transferReceive']
print(basicRate)
print(cashBuy)
print(cashSell)
print(transferSend)
print(transferReceive)

<class 'str'>
1,009.45
974.73
1,001.81
982.37


In [27]:
import pymysql 
 
conn = pymysql.connect(host='localhost', user='root', password='1234', db='jpy', charset='utf8') 

In [28]:
cur = conn.cursor()
sql = """ INSERT INTO money2(basicRate, cashBuy, cashSell, transferSend, transferReceive)
        VALUES(%s,%s,%s,%s,%s)"""

In [29]:
cur.execute(sql, (basicRate,cashBuy, cashSell, transferSend, transferReceive))

1

In [30]:
# conn 닫기
conn.commit()
conn.close()